Compare House TCJA total change in liability when using default weights vs. those provided by Ernie Tedeschi. 

Uses behavior file provided by Matt Jensen.

In [1]:
from taxcalc import *
from taxcalc.utils import *

In [2]:
# Load Ernie Tedeschi's weights file.
tedeschi_weights = pd.read_csv('~/cps_weights_2026.csv')

Something to do with behavior? Doesn't calculate `combined` without `Behavior.response()`.

In [21]:
Records.cps_constructor().s006

array([ 230.01666667,  194.6       ,  245.27666667, ...,  113.12666667,
         96.25666667,  113.12666667])

In [33]:
def total_combined_liability(calc):
    # combined is combined tax liability while s006 is weight
    return (calc.records.combined * calc.records.s006).sum()

In [34]:
def print_combined_liability(baseline_calc, reform_calc):
    # Calculate and print liabilities.
    baseline = total_combined_liability(baseline_calc)
    reformed = total_combined_liability(reform_calc)
    diff = reformed - baseline
    print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
    print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
    print ('-' * 47)
    print ('Difference: {:35.2f}'.format(diff))

In [40]:
def new_test(replace_records_tedeschi_weights,
             replace_calc_tedeschi_weights):
    # Initiate baseline calculator
    recs = Records.cps_constructor()
    if (replace_records_tedeschi_weights):
        recs.s006 = tedeschi_weights.s006
    calc = Calculator(records=recs, policy=Policy())
    if (replace_calc_tedeschi_weights):
        calc.s006 = tedeschi_weights.s006
    # Advance and calculate.
    calc.advance_to_year(2018)
    calc.calc_all()
    # Initiate calculator to apply reforms to
    reform = Calculator.read_json_param_objects('TCJA_House_Amended.json',
                                                'JCT_Behavior.json')
    pol = Policy()
    pol.implement_reform(reform['policy'])
    con = Consumption()
    con.update_consumption(reform['consumption'])
    beh = Behavior()
    beh.update_behavior(reform['behavior'])
    calc_x = Calculator(records=recs, policy=pol,
                        consumption=con, behavior=beh)
    if (replace_calc_tedeschi_weights):
        calc_x.s006 = tedeschi_weights.s006
    calc_x.advance_to_year(2018)
    calc_response = Behavior.response(calc, calc_x)
    print_combined_liability(calc, calc_response)

In [42]:
new_test(replace_records_tedeschi_weights = True,
         replace_calc_tedeschi_weights = True)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2458101656483.74
-----------------------------------------------
Difference:                     -62968854965.90


In [45]:
new_test(replace_records_tedeschi_weights = False,
         replace_calc_tedeschi_weights = False)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2458101656483.74
-----------------------------------------------
Difference:                     -62968854965.90


In [39]:
new_test(use_tedeschi_weights = False)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
Combined Liability - Baseline: 2521070511449.64
Combined Liability - Reform:   2458101656483.74
-----------------------------------------------
Difference:                     -62968854965.90


In [24]:
def test(use_tedeschi_weights):
    # Initiate baseline calculator
    recs = Records.cps_constructor()
    if (use_tedeschi_weights):
        recs.s006 = tedeschi_weights.s006
    calc = Calculator(records=recs, policy=Policy())
    # Initiate calculator to apply reforms to
    reform = Calculator.read_json_param_objects('TCJA_House_Amended.json',
                                                'JCT_Behavior.json')
    pol = Policy()
    pol.implement_reform(reform['policy'])
    con = Consumption()
    con.update_consumption(reform['consumption'])
    beh = Behavior()
    beh.update_behavior(reform['behavior'])
    calc_x = Calculator(records=recs, policy=pol,
                        consumption=con, behavior=beh)
    calc_x.advance_to_year(2018)
    calc_response = Behavior.response(calc, calc_x)
    # Calculate and print liabilities.
    baseline = (calc.records.combined * calc.records.s006).sum()  # combined is combined tax liability while s006 is weight
    reformed = (calc_response.records.combined * calc_response.records.s006).sum()
    diff = reformed - baseline
    print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
    print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
    print ('-' * 47)
    print ('Difference: {:35.2f}'.format(diff))

In [31]:
test(use_tedeschi_weights = True)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


AssertionError: 

In [26]:
test(use_tedeschi_weights = False)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


AssertionError: 

In [ ]:
# Initiate calculator to apply reforms to
recs_x = Records.cps_constructor()
calc_x = Calculator(records=recs_x, policy=Policy())
# Replace weights attribute of calc_x.
calc_x.records.s006 = tedeschi_weights.s006

In [ ]:
reforms = calc_x.read_json_param_objects('TCJA_House_old.json', 'JCT_Behavior.json')

You can specify your reforms in a JSON file and, using the `read_json_param_objects` method, create a dictionary containing both policy reforms and behavioral assumptions

The policy reform and any behavioral assumptions need to be in separate JSON files.

Here is [more information](https://github.com/open-source-economics/Tax-Calculator/blob/master/taxcalc/reforms/REFORMS.md) on creating reform files.

In [ ]:
calc_x.policy.implement_reform(reforms['policy'])
calc_x.consumption.update_consumption(reforms['consumption'])
calc_x.advance_to_year(2018)

### Behavioral response

A dictionary is also used to implement behavioral reforms. The only difference is you must then pass the baseline and reform calculators into the response method of the behavioral class. This method calculates the change in tax liabilities and then, using the specified elasticities, returns a new calculator object that accounts for any behavioral change.

In [ ]:
calc_x.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc, calc_x)


### Without Tedeschi weights

In [ ]:
# Initiate baseline calculator
recs = Records.cps_constructor()
calc = Calculator(records=recs, policy=Policy())
# Replace weights attribute of calc.
#calc.records.s006 = tedeschi_weights.s006
# Advance and calculate.
calc.advance_to_year(2018)
calc.calc_all()

# Initiate calculator to apply reforms to
recs_x = Records.cps_constructor()
calc_x = Calculator(records=recs_x, policy=Policy())
# Replace weights attribute of calc_x.
#calc_x.records.s006 = tedeschi_weights.s006

calc_x.policy.implement_reform(reforms['policy'])
calc_x.consumption.update_consumption(reforms['consumption'])
calc_x.advance_to_year(2018)

calc_x.behavior.update_behavior(reforms['behavior'])
calc_response = Behavior.response(calc, calc_x)

### Viewing the results

In [ ]:
from taxcalc.utils import *

#### Analyzing Individual Variables

Individual variables are attributes of the records class and can therefore be accessed using a simple dot notation

In [ ]:
baseline = (calc.records.combined * calc.records.s006).sum()  # combined is combined tax liability while s006 is weight
reformed = (calc_response.records.combined * calc_response.records.s006).sum()
diff = reformed - baseline
print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
print ('-' * 47)
print ('Difference: {:35.2f}'.format(diff))

#### Diagnostic Table

Diagnostic tables are the most straight forward methods of evaluation. They simply show aggregate values for a given calculator

In [ ]:
create_diagnostic_table(calc)

In [ ]:
create_diagnostic_table(calc_response)

#### Distribution Table

The distribution table shows the same information as the diagnostic table, but broken down by income bin or decile. You can view the results as either the weighted average or the weighted sum in each bin

In [ ]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_avg')

In [ ]:
create_distribution_table(calc_response.records, groupby='weighted_deciles',
                          income_measure='expanded_income', result_type='weighted_sum')

#### Differences Table

The differences table displays the difference between your baseline and refoms. You can also group the results by decile or income bin.

In [ ]:
create_difference_table(calc.records, calc_response.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

#### Plotting

You can use built in methods to get MTR and ATR plots. Each one is returned as a simple Bokeh figure that you can then add to as desired

In [ ]:
from bokeh.io import show, output_notebook
output_notebook()

In [ ]:
mtr_plot_data = mtr_graph_data(calc, calc_response)

In [ ]:
show(xtr_graph_plot(mtr_plot_data))

In [ ]:
atr_plot_data = atr_graph_data(calc, calc_response)

In [ ]:
show(xtr_graph_plot(atr_plot_data))

#### Multiyear diagnostic tables

You can also produce diagnostic tables up to 2026

In [ ]:
multiyear_diagnostic_table(calc, num_years=9)

In [ ]:
multiyear_diagnostic_table(calc_response, num_years=9)

#### Reporting your changes

To display what reforms you included in a way that is easy for humans to read, you can use the `reform_documentation` method. It will print out all of the policy parameters you've specified with a short description and their default and current values


_Note: this feature is not yet available in the taxcalc package. You must use the source code to access it. It will be available in the next taxcalc release._

In [ ]:
print(calc_response.reform_documentation(reforms))

One thing I didn't cover is how to extract the marginal tax rates. All you need to do is call the `Calculator.mtr()` method. It will return MTR for individual income tax, payroll tax, and individaul income tax + payroll tax

In [ ]:
mtr_payroll, mtr_income, mtr_combined = calc.mtr()

In [ ]:
mtr_payroll

In [ ]:
mtr_income

In [ ]:
mtr_combined